In [2]:
import pandas as pd
import numpy as np

In [3]:
stints = pd.read_csv("../data/pbp_stints.csv")

In [ ]:
#We do NOT have that many stints where both teams have 5 players each, oh well.

In [4]:
filtered_stints = stints.loc[stints.player_id.map(stints.player_id.value_counts()) > 200]

In [5]:
idx = filtered_stints.groupby("stint_id")["points"].idxmax()
points_max = filtered_stints.loc[idx, ["stint_id", "player_id", "points"]]
(points_max.player_id.value_counts() / filtered_stints.player_id.value_counts()).sort_values(ascending=False).head(20)

player_id
curryst01    0.284043
gilgesh01    0.274562
doncilu01    0.268381
duranke01    0.267804
mitchdo01    0.266442
edwaran01    0.265221
embiijo01    0.260727
antetgi01    0.255926
jamesle01    0.255711
georgpa01    0.254018
moranja01    0.253652
jacksfr01    0.252778
tatumja01    0.252358
foxde01      0.248173
leonaka01    0.246674
lavinza01    0.245509
grantje01    0.245497
thomaca02    0.241327
irvinky01    0.240472
jokicni01    0.238244
Name: count, dtype: float64

In [6]:
#assists
ast_idx = filtered_stints.groupby("stint_id")["assists"].idxmax()
assist_max = filtered_stints.loc[ast_idx, ["stint_id", "player_id", "assists"]]
(assist_max.player_id.value_counts() / filtered_stints.player_id.value_counts()).sort_values(ascending=False).head(20)

player_id
halibty01    0.396812
youngtr01    0.392610
doncilu01    0.387897
paulch01     0.362602
hardeja01    0.351783
moranja01    0.334661
jokicni01    0.328247
porteke02    0.323442
lillada01    0.312332
cunnica01    0.311828
walljo01     0.308271
jonestr01    0.303953
jamesle01    0.299926
garlada01    0.296607
micicva01    0.293750
simmobe01    0.292683
ballla01     0.290015
hayeski01    0.288961
greendr01    0.287311
murrade01    0.283300
Name: count, dtype: float64

In [7]:
#shots
shot_idx = filtered_stints.groupby("stint_id")["FGA"].idxmax()
shot_max = filtered_stints.loc[shot_idx, ["stint_id", "player_id", "FGA"]]
(shot_max.player_id.value_counts() / filtered_stints.player_id.value_counts()).sort_values(ascending=False).head(20)

player_id
doncilu01    0.326172
mitchdo01    0.313528
curryst01    0.310284
edwaran01    0.302346
cunnica01    0.297280
wembavi01    0.294542
foxde01      0.292660
gilgesh01    0.287132
duranke01    0.287092
tatumja01    0.286388
jamesle01    0.277819
bealbr01     0.275728
bookede01    0.275100
moranja01    0.274900
irvinky01    0.274501
ingrabr01    0.273058
millebr02    0.272441
thomaca02    0.268477
kuzmaky01    0.268465
murraja01    0.268035
Name: count, dtype: float64

In [8]:
#rebounds
rebound_idx = filtered_stints.groupby("stint_id")["rebounds"].idxmax()
rebound_max = filtered_stints.loc[rebound_idx, ["stint_id", "player_id", "rebounds"]]
(rebound_max.player_id.value_counts() / filtered_stints.player_id.value_counts()).sort_values(ascending=False).head(20)

player_id
goberru01    0.365746
drumman01    0.352829
kanteen01    0.352668
valanjo01    0.332966
yurtsom01    0.328358
howardw01    0.328194
capelca01    0.327978
sabondo01    0.326316
jokicni01    0.325755
brownmo01    0.323432
nurkiju01    0.320973
davisan02    0.312650
kesslwa01    0.308934
wembavi01    0.305870
antetgi01    0.305050
allenja01    0.304836
durenja01    0.302303
garubus01    0.299180
aytonde01    0.297625
adamsst01    0.291807
Name: count, dtype: float64

In [9]:
stints = pd.read_csv("../data/pbp_stints.csv")
lineups = stints.groupby("stint_id")["player_id"].agg(list).reset_index()

In [ ]:
#Okay, figuring out the new +/- now.

stints = pd.read_csv("../data/pbp_stints.csv")
lineups = stints.groupby("stint_id")["player_id"].agg(list).reset_index()


def teammate_plus_minus(stint_id, player_id, team):
    players = stints.loc[stints.stint_id == stint_id]
    team_players = players.loc[players.home == team]
    team_players = team_players.loc[team_players.player_id != player_id]
    #Now we loop through our lineups, find the stint_ids that match the lineup we're looking for, and calculate their plus minus
    plusminus = 0
    seconds = 0
    if len(team_players.player_id) < 2:
        #Should have at least 2 teammates to consider
        return plusminus, seconds
    for _, row in lineups.iterrows():
        lineup = row.player_id
        if all(x in lineup for x in team_players.player_id):
            stint_data = stints.loc[stints.stint_id == row.stint_id]
            stint_data = stint_data.loc[stint_data.home == team]
            plusminus += stint_data["+/-"].mean() #technically they'll all be the same anyway
            seconds += stint_data.seconds.mean()
    return plusminus, seconds

def opponent_plus_minus(stint_id, team):
    players = stints.loc[stints.stint_id == stint_id]
    opp_players = players.loc[players.home != team]
    plusminus = 0
    seconds = 0
    if len(opp_players.player_id) < 3:
        #Should have at least 3 opponents to consider
        return plusminus, seconds
    for _, row in lineups.iterrows():
        lineup = row.player_id
        if all(x in lineup for x in opp_players.player_id):
            stint_data = stints.loc[stints.stint_id == row.stint_id]
            stint_data = stint_data.loc[stint_data.home != team]
            plusminus += stint_data["+/-"].mean() #technically they'll all be the same anyway
            seconds += stint_data.seconds.mean()
    return plusminus, seconds


for index, row in stints.iterrows():
    stint_id = row.stint_id
    player_id = row.player_id
    team = row.home
    tm_plusminus, tm_seconds = teammate_plus_minus(stint_id, player_id, team)
    op_plusminus, op_seconds = opponent_plus_minus(stint_id, team)
    stints.loc[index, "New +/-"] = row["+/-"] - ((tm_plusminus / tm_seconds * row.seconds if tm_seconds > 0 else 0) - (op_plusminus / op_seconds * row.seconds if op_seconds > 0 else 0))/2
    if index % 1000 == 0:
        print(f"Processed {index} rows")
    

In [10]:
lineups["player_set"] = lineups.player_id.apply(frozenset)
lineup_dict = dict(zip(lineups.stint_id, lineups.player_set))
stint_info = (
    stints.groupby("stint_id")
          .agg(pm=("+/-","mean"), sec=("seconds","mean"))
)


In [12]:
from collections import defaultdict

player_to_stints = defaultdict(set)
for stint_id, pset in lineup_dict.items():
    for p in pset:
        player_to_stints[p].add(stint_id)

def stints_containing(players):
    sets = [player_to_stints[p] for p in players]
    return set.intersection(*sets)
stint_team = stints.groupby("stint_id")["home"].first()

In [13]:
def teammate_plus_minus_fast(stint_id, player_id, team):
    players = lineup_dict[stint_id]
    team_players = [p for p in players if stint_team.loc[stint_id] == team and p != player_id]

    if len(team_players) < 2:
        return 0.0, 0.0

    matching = stints_containing(team_players)
    if not matching:
        return 0.0, 0.0

    pm = stint_info.loc[list(matching), "pm"].sum()
    sec = stint_info.loc[list(matching), "sec"].sum()
    return pm, sec


def opponent_plus_minus_fast(stint_id, team):
    players = lineup_dict[stint_id]
    opp_players = [p for p in players if stint_team.loc[stint_id] != team]

    if len(opp_players) < 3:
        return 0.0, 0.0

    matching = stints_containing(opp_players)
    if not matching:
        return 0.0, 0.0

    pm = stint_info.loc[list(matching), "pm"].sum()
    sec = stint_info.loc[list(matching), "sec"].sum()
    return pm, sec

In [14]:
rows = []
for index, row in stints.iterrows():
    stint_id = row.stint_id
    player_id = row.player_id
    team = row.home

    tm_pm, tm_sec = teammate_plus_minus_fast(stint_id, player_id, team)
    op_pm, op_sec = opponent_plus_minus_fast(stint_id, team)

    adj = row["+/-"]
    if tm_sec > 0:
        adj -= (tm_pm / tm_sec * row.seconds) / 2
    if op_sec > 0:
        adj += (op_pm / op_sec * row.seconds) / 2

    rows.append(adj)


In [18]:
stints["new +/-"] = rows

In [20]:
stints.to_csv("../data/pbp_stints_plusminus.csv", index=False)